In [30]:
pip install shap

  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/f5/fc/e81722d6bec4fcba46e46ef895eddaeab0027ac71e78fc35ef351fac5fe4/shap-0.43.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/447.3 kB ? eta -:--:--
   -- ------------------------------------- 30.7/447.3 kB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 61.4/447.3 kB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 225.3/447.3 kB 2.3 MB/s eta 0:00:01
   ---------------------------------- ----- 389.1/447.3 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 447.3/447.3 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [130]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01
from sklearn.preprocessing import OneHotEncoder

import sklearn.tree as tree

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

In [217]:
# Reading in the CSV file
dia_data = pd.read_csv("data/diabetic_data.csv")
X = dia_data.iloc[:, :-1]  # All columns except the last one
Y = dia_data.iloc[:, -1]   # Last column
#train 70% temp 30%
Xtr, Xtemp, Ytr, Ytemp = train_test_split(X, Y, test_size=0.3, random_state=seed)


# validation: 10% Test: 20%
Xva, Xtest, Yva, Ytest = train_test_split(Xtemp, Ytemp, test_size=0.67, random_state=seed)

print(Xtr.shape, Ytr.shape)
print(Xva.shape, Yva.shape)

# View the first 5 rows
print(Xtr.columns[17:33])

(71236, 49) (71236,)
(10074, 49) (10074,)
Index(['number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide'],
      dtype='object')


In [218]:
Xtr

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
47866,146752038,25174899,AfricanAmerican,Female,[10-20),?,1,1,7,3,...,No,No,Down,No,No,No,No,No,Ch,Yes
24673,83354610,298521,AfricanAmerican,Female,[90-100),?,1,14,7,3,...,No,No,No,No,No,No,No,No,No,No
61033,170636856,77827617,Caucasian,Male,[80-90),[75-100),2,6,1,4,...,No,No,No,No,No,No,No,No,No,Yes
74444,221787444,38516562,?,Male,[80-90),?,1,1,7,1,...,No,No,No,No,No,No,No,No,No,No
56971,163020096,46248840,AfricanAmerican,Female,[30-40),?,1,1,7,10,...,No,No,Down,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89460,289025640,50693553,Caucasian,Male,[80-90),?,1,6,7,9,...,No,No,No,No,No,No,No,No,Ch,Yes
60620,169681170,66287709,Caucasian,Male,[50-60),?,5,1,1,3,...,No,No,No,No,No,No,No,No,No,Yes
34086,106962006,24745671,Caucasian,Female,[80-90),?,2,1,4,5,...,No,No,Steady,No,No,No,No,No,No,Yes
58067,164976492,59602239,Caucasian,Female,[80-90),?,1,1,7,3,...,No,No,No,No,No,No,No,No,No,No


In [219]:
missing_values = Xtr['glyburide'].isna().sum()
# Print or display the number of missing values in each column
print(missing_values)

0


In [220]:
Xtr.iloc[:,17:33]
print(Xtr.columns[18:33])

Index(['diag_1', 'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum',
       'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide'],
      dtype='object')


In [221]:
Xtr['glyburide'].value_counts()

glyburide
No        63876
Steady     6419
Up          554
Down        387
Name: count, dtype: int64

In [222]:
#missing_values = Xtr['glyburide_Up'].isna().sum()
#missing_values1 = Xtr['glyburide_Down'].isna().sum()
#missing_values2 = Xtr['glyburide_Steady'].isna().sum()
#missing_values3 = Xtr['glyburide_No'].isna().sum()

# Print or display the number of missing values in each column
#print(missing_values, missing_values1, missing_values2, missing_values3)
#print(Xtr['glyburide_Up'].value_counts())
#print(Xtr['glyburide_Down'].value_counts())
#print(Xtr['glyburide_Steady'].value_counts())
#print(Xtr['glyburide_No'].value_counts())

In [223]:
#dia_1, dia_2 and dia_3 have too many distinct number so should not do one hot encoding 
categorical_columns =['max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide']
#, 'number_diagnoses',
#       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
#       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
#       'glyburide', 'tolbutamide']
Xtr_categorical = Xtr[categorical_columns]
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
Xtr_encoded = encoder.fit_transform(Xtr_categorical)
Xtr_encoded_df = pd.DataFrame(Xtr_encoded, columns=encoder.get_feature_names_out(categorical_columns))
Xtr = pd.concat([Xtr.reset_index(drop=True), Xtr_encoded_df], axis='columns')
#Xtr = pd.concat([Xtr, Xtr_encoded_df], axis=1)
Xtr = Xtr.drop(categorical_columns, axis=1)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [224]:
Xtr

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,glipizide_Down,glipizide_No,glipizide_Steady,glipizide_Up,glyburide_Down,glyburide_No,glyburide_Steady,glyburide_Up,tolbutamide_No,tolbutamide_Steady
0,146752038,25174899,AfricanAmerican,Female,[10-20),?,1,1,7,3,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,83354610,298521,AfricanAmerican,Female,[90-100),?,1,14,7,3,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,170636856,77827617,Caucasian,Male,[80-90),[75-100),2,6,1,4,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,221787444,38516562,?,Male,[80-90),?,1,1,7,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,163020096,46248840,AfricanAmerican,Female,[30-40),?,1,1,7,10,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,289025640,50693553,Caucasian,Male,[80-90),?,1,6,7,9,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
71232,169681170,66287709,Caucasian,Male,[50-60),?,5,1,1,3,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
71233,106962006,24745671,Caucasian,Female,[80-90),?,2,1,4,5,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
71234,164976492,59602239,Caucasian,Female,[80-90),?,1,1,7,3,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [225]:
Xtr.columns[50:]

Index(['repaglinide_Down', 'repaglinide_No', 'repaglinide_Steady',
       'repaglinide_Up', 'nateglinide_Down', 'nateglinide_No',
       'nateglinide_Steady', 'nateglinide_Up', 'chlorpropamide_Down',
       'chlorpropamide_No', 'chlorpropamide_Steady', 'chlorpropamide_Up',
       'glimepiride_Down', 'glimepiride_No', 'glimepiride_Steady',
       'glimepiride_Up', 'acetohexamide_No', 'acetohexamide_Steady',
       'glipizide_Down', 'glipizide_No', 'glipizide_Steady', 'glipizide_Up',
       'glyburide_Down', 'glyburide_No', 'glyburide_Steady', 'glyburide_Up',
       'tolbutamide_No', 'tolbutamide_Steady'],
      dtype='object')

In [236]:
missing_values = Xtr["diag_3"].value_counts(dropna=False)
# Print or display the number of missing values in each column
print(missing_values)

diag_3
250    8110
401    5722
276    3648
428    3271
427    2797
       ... 
226       1
186       1
953       1
417       1
989       1
Name: count, Length: 746, dtype: int64


In [247]:
# Get the value counts for each unique value in "diag_3"
value_counts = Xtr["diag_3"].value_counts(dropna=False)

# Identify values with a frequency of 1
values_to_group = value_counts[value_counts < 20].index

# Replace values with a frequency of 1 with a common label (e.g., 'Other')
Xtr["diag_3_grouped"] = Xtr["diag_3"].replace(values_to_group, 'Other')

# Print or display the updated value counts
updated_value_counts = Xtr["diag_3_grouped"].value_counts(dropna=False)
print(updated_value_counts)


diag_3_grouped
250    8110
401    5722
276    3648
428    3271
427    2797
       ... 
723      20
659      20
V65      20
289      20
369      20
Name: count, Length: 252, dtype: int64


In [227]:
# Assuming df is your DataFrame and columns_list contains the columns you want to process
columns_list = ['glipizide_Down', 'glipizide_No', 'glipizide_Steady', 'glipizide_Up',
                'glyburide_Down', 'glyburide_No', 'glyburide_Steady', 'glyburide_Up',
                'tolbutamide_No', 'tolbutamide_Steady']

# Replace NaN values with mode in specified columns
for column in columns_list:
    mode_value = Xtr[column].mode()[0]
    Xtr[column].fillna(mode_value, inplace=True)
